## Animating the network-area-diagram: bringing the grid to life!
  - This notebook demonstrates how to **visualize network states over time**.
  - We will build a time-indexed dataset and drive a diagram with a time slider.

#### 1. Load the network snapshots
- We create 2 snapshots based on the IEEE 300 network.
- We build a timestamp dataframe based on
   - p1, p2 values
   - connected1, connected2 values

In [ ]:
import pypowsybl.network as pn
import pypowsybl.loadflow as plf
import pandas as pd

network_changes = pd.DataFrame({
    'timestamp': ['1700', '1730', '1800'],
    'b1_l_increase': [0, 200, 400],
    'b76_g_increase': [0, 200, 400],
    'l5_9_connection': [True, False, False]
})

rows = []
for row in network_changes.itertuples(index=True, name="Row"):
    
    # faking the loading of real snapshots
    network = pn.create_ieee300()
    network.update_loads(id='B1-L', p0 = 90 + row.b1_l_increase)
    network.update_generators(id='B76-G', target_p = row.b76_g_increase)
    network.update_branches(id='L5-9-1', connected1 = row.l5_9_connection)

    plf.run_ac(network)
    
    branches = network.get_branches(attributes=["p1", "p2", "connected1", "connected2"])
    branches = branches.reset_index()
    timestamp_df = pd.DataFrame({
        'branch_id': branches['id'],
        'timestamp': pd.to_datetime(row.timestamp, format="%H%M"),
        'value1': branches['p1'].fillna(0.0),
        'value2': branches['p2'].fillna(0.0),
        'connected1': branches['connected1'],
        'connected2': branches['connected2']
    })
    rows.append(timestamp_df)

time_series_df = pd.concat(rows, ignore_index=True)
time_series_df

- searching the branches whose connection state changes during the studied time slot

In [ ]:
changing_c1 = time_series_df.groupby('branch_id')['connected1'].nunique() > 1
changing_c2 = time_series_df.groupby('branch_id')['connected2'].nunique() > 1
changing_branches = (changing_c1 | changing_c2)
changing_branches[changing_branches].index.tolist()

#### 2. Displaying the results
- Again, we use the pseudo-geographical fixed nad positions
- Two limitations:
    - the time slider is only available in nad_explorer widget so far
    - only active power values are updated so far 

##### Displaying the nad_explorer widget for the full network
- Select VL5 (or zoom on it) to see the line disconnection and the increase of active powers near VL1.
- Select VL76 (or zoom on it) to see the increase of active powers around it.

In [ ]:
from pypowsybl_jupyter import nad_explorer

nad_explorer(network, depth=2, time_series_data=time_series_df)